# 채권 민감도 지표

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
import QuantLib as ql
options = webdriver.ChromeOptions()
options.add_argument('headless')

# 금리 커브 모델 가져오기

In [3]:
def GET_DATE():
    driver = webdriver.Chrome('C:\it\python_workspace\chromedriver', options=options)
    driver.get("https://www.wsj.com/market-data/bonds")
    html = driver.page_source    
    soup = BeautifulSoup(html, 'html.parser')
    data = soup.find("span", class_="WSJBase--card__timestamp--3F2HxyAE")
    date = data.text
    date = date.split(' ')[3]
    date = datetime.datetime.strptime(date, "%m/%d/%y").date()
    return date

In [4]:
def GET_QUOTE(reference_date):
    driver = webdriver.Chrome('C:\it\python_workspace\chromedriver', options=options)
    tenors = ['01M', '03M', '06M', '01Y', '02Y', '03Y', '05Y', '07Y', '10Y', '30Y']
    
    # Create Empty Lists
    maturities = []
    days = []
    prices = []
    coupons = []
    
    # Get Market Information    
    for i, tenor in enumerate(tenors):
        driver.get("https://quotes.wsj.com/bond/BX/TMUBMUSD" + tenor + "?mod=md_bond_overview_quote")
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        # Price
        if i <= 3:
            data_src = soup.find("span", id="quote_val")
            price = data_src.text
            price = float(price[:-1])
        else:
            data_src = soup.find("span", id="price_quote_val")
            price = data_src.text
            price = price.split()
            price1 = float(price[0])
            price = price[1].split('/')
            price2 = float(price[0])
            price3 = float(price[1])            
            price = price1 + (price2 / price3)  
            
        data_src2 = soup.find_all("span", class_="data_data")
         # Coupon
        coupon = data_src2[2].text
        if coupon != '':
            coupon = float(coupon[:-1])
        else:
            coupon = 0.0
        
        # Maturity Date
        maturity = data_src2[3].text
        maturity = datetime.datetime.strptime(maturity, '%m/%d/%y').date()
        
        # Send to Lists
        days.append((maturity - reference_date).days)
        prices.append(price)
        coupons.append(coupon)
        maturities.append(maturity)
    
    # Create DataFrame
    df = pd.DataFrame([maturities, days, prices, coupons]).transpose()
    headers = ['maturity', 'days', 'price', 'coupon']
    df.columns = headers
    df.set_index('maturity', inplace=True)
    
    return df

In [5]:
def TREASURY_CURVE(date, quote):
    
    # Divide Quotes
    tbill = quote[0:4]
    tbond = quote[4:]
    # 평가일 설정
    eval_date = ql.Date(date.day, date.month, date.year)
    ql.Settings.instance().evaluationDate = eval_date
    # 마켓 컨벤션 설정 #달력,금리설정일,이자계산방식,정산일자선정방식,명목금액,이자정산주기
    calendar = ql.UnitedStates()
    convention = ql.ModifiedFollowing
    day_counter = ql.ActualActual()
    end_of_month = True
    fixing_days = 1
    face_amount = 100
    coupon_frequency = ql.Period(ql.Semiannual)
    #Deposit Rate Helper
    #DepositRateHelper-무이표채에적용,
    #FixedRateBondHelper-이표채에적용
    bill_helpers = [ql.DepositRateHelper(ql.QuoteHandle(ql.SimpleQuote(r/100.0)),
                                        ql.Period(m, ql.Days),
                                        fixing_days,
                                        calendar,
                                        convention,
                                        end_of_month,
                                        day_counter)
                   for r, m in zip(tbill['price'], tbill['days'])]
                #for 문으로 무이표채 금리와 잔존일수 데이터 입력
    # Construct Treasury Bond Helpers [ Fixed Bond Rate Helper ]
    bond_helpers = []
    for p, c, m in zip(tbond['price'], tbond['coupon'], tbond['days']):
        termination_date = eval_date + ql.Period(m, ql.Days)
        schedule = ql.Schedule(eval_date,
                              termination_date,
                              coupon_frequency,
                              calendar,
                              convention,
                              convention,
                              ql.DateGeneration.Backward,
                              end_of_month)
        bond_helper = ql.FixedRateBondHelper(ql.QuoteHandle(ql.SimpleQuote(p)),
                                            fixing_days,
                                             face_amount,
                                             schedule,
                                             [c/100.0],
                                             day_counter,
                                             convention)
        bond_helpers.append(bond_helper)
        # FixedRateBondHelper - 이표채 이자지급스케쥴 생성
    # 헬퍼 결합(무이표채 + 이표채)
    rate_helper = bill_helpers + bond_helpers
    # 헬퍼를 커브 모듈에 태우기
    yc_linearzero = ql.PiecewiseLinearZero(eval_date, rate_helper, day_counter)
    return yc_linearzero

# 민감도 구하기

In [8]:
# Import Reference Curve Date

ref_date = GET_DATE()
quote = GET_QUOTE(ref_date)
curve = TREASURY_CURVE(ref_date, quote)

In [9]:
# Convert into Engine
spotCurveHandle = ql.YieldTermStructureHandle(curve)
bondEngine = ql.DiscountingBondEngine(spotCurveHandle)

채권 정보를 받아 fixedRateBond에 입력

In [12]:
# Treasury Bond Specification
issueDate = ql.Date(15,11,2019)
maturityDate = ql.Date(15,11,2029)
tenor = ql.Period(ql.Semiannual)
calendar = ql.UnitedStates()
convention = ql.ModifiedFollowing
dateGeneration = ql.DateGeneration.Backward
monthEnd = False
schedule = ql.Schedule(issueDate,
                      maturityDate,
                      tenor,
                      calendar,
                      convention,
                      convention,
                      dateGeneration,
                      monthEnd)
dayCount = ql.ActualActual()
couponRate = [0.0175]
settlementDays = 1
faceValue = 100

fixedRateBond = ql.FixedRateBond(settlementDays, faceValue, schedule, couponRate, dayCount)

In [13]:
# Couduct Pricing
fixedRateBond.setPricingEngine(bondEngine)

# BondFunctions -> 듀레이션과 컨벡시티 계산

In [15]:
# Calculate YTM

# fixedRateBond() 와 interestRate를 인자로 삼는다.

targetPrice = fixedRateBond.cleanPrice()
ytm = ql.InterestRate(fixedRateBond.bondYield(targetPrice, dayCount, ql.Compounded, ql.Semiannual),
                     dayCount,
                     ql.Compounded,
                     ql.Semiannual)
print("Yield to Maturity = {:.4%}".format(ytm.rate()))

# Calculate Duration & Convexity
print("Duration = {}".format(round(ql.BondFunctions.duration(fixedRateBond,ytm),4)))
print("Convexity = {}".format(round(ql.BondFunctions.convexity(fixedRateBond,ytm),4)))

Yield to Maturity = 1.2803%
Duration = 7.8386
Convexity = 68.5523
